In [1]:
import torch
from torch.autograd import Variable
import cv2
import sys
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

In [2]:
def detect(frame, net, transform): 
    #frame-image on which detect function will be applied, (original frame not grayscale),,,net-- ssd nn network,,, transform---transform applied to image to set right format
    # will return frame with objects along boxes
    
    height, width = frame.shape[:2] #return h, w, channels(0,1,2---RGB)
    
    # convert into torch variable using 4 transformations
    # 1. apply transform transformation to set right dimensions
    # 2. convert transformed frame from numpy to torch tensor
    # 3. add fake dimensions to torch tensor corresponding to batch
    # 4. to convert tensor into torch variable(contains both tensor and gradients) --- feed the image to NN
    
    frame_t = transform(frame)[0] # return 2 elements but we get only one
    x = torch.from_numpy(frame_t).permute(2,0,1) # transformation no 2,then, convert RBG into GRB
    # NN accpets only batches of input
    
    # x.unsqueeze(0) # no 3
    x = Variable(x.unsqueeze(0)) # No 4
    y = net(x) # feed x to NN, return o/p y   
    detections = y.data # to get values of the output
    scale = torch.Tensor([width, height, width, height]) # normalize the values
    
    # detection tensor contains 4 elements --> detections = [batch, number_of_classes, number_of_occurences_of_the_class, [score, x0, y0, x1, y1]] for each occurence we will get score and top-left and bottom-right coordinates, scores(low to high) if score is low than thrershold then it will not detect object occurence
    for i in range(detections.size(1)): #iterate through all the classes
        j = 0 #occurence of the class
        while detections[0, i, j, 0] >= 0.6: # 0 is score occurence j of class i, 0 is score
            pt = (detections[0,i,j,1:] * scale).numpy() # scale to apply normalization, normalized coordinates 
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255,0,0), 2)
            cv2.putText(frame, labelmap[i-1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            j += 1
    return frame   

In [3]:
#Creating an SSD Neural Network

net = build_ssd('test') #argument is the face, train and test face
net.load_state_dict(torch.load('ssd300_map_77.43_v2.pth', map_location = lambda storage, loc:storage))

C:\Users\Dell\Downloads\Single Shot  MultiBox Detector-20190726T185953Z-001\Single Shot  MultiBox Detector\ssd.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
C:\Users\Dell\Downloads\Single Shot  MultiBox Detector-20190726T185953Z-001\Single Shot  MultiBox Detector\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [4]:
# Creating the transformation

transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0)) #normalize color values

In [5]:
#doing object detecion on a video frame by frame

reader = imageio.get_reader('PARIS WALK _ Luxembourg Gardens - Magnificent Paris Park _ France_Trim.mp4')
fps = reader.get_meta_data()['fps']#get number of frame per second
writer = imageio.get_writer('output.mp4', fps = fps) #create op video with same fps rate

for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame) #append frame to writer video
    print(i) #number of processed frame while detection
writer.close()
    

C:\Users\Dell\Downloads\Single Shot  MultiBox Detector-20190726T185953Z-001\Single Shot  MultiBox Detector\ssd.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.softmax(conf.view(-1, self.num_classes)),  # conf preds


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27